https://github.com/yu4u/mixup-generator


In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

In [0]:
ROOT = '/content/drive/My Drive/19 Fall/CSE 569/Project'

In [64]:
X_train = pickle.load(open(ROOT + '/cifar/X_train', mode='rb'))
print(X_train.shape)
X_test = pickle.load(open(ROOT + '/cifar/X_test', mode='rb'))
print(X_test.shape)
X_val = pickle.load(open(ROOT + '/cifar/X_val', mode='rb'))
print(X_val.shape)
y_train = pickle.load(open(ROOT + '/cifar/y_train', mode='rb'))
print(y_train.shape)
y_test = pickle.load(open(ROOT + '/cifar/y_test', mode='rb'))
print(y_test.shape)
y_val = pickle.load(open(ROOT + '/cifar/y_val', mode='rb'))
print(y_val.shape)

(32000, 32, 32, 3)
(10000, 32, 32, 3)
(8000, 32, 32, 3)
(32000, 10)
(10000, 10)
(8000, 10)


In [0]:

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.datasets import cifar10
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time


In [0]:
# Functions for plots
def plot_model_history(model_history):
    fig, axs = plt.subplots(1, 2,figsize=(10, 5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'validation'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'validation'], loc='best')
    plt.show()

In [67]:
# Define the model
num_filters = 32

model = Sequential()
model.add(Conv2D(num_filters, (3,3), padding='same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Summarize the model
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 64)       

In [0]:
import numpy as np


class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, num_unlabled=28, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen
        self.num_unlabled = num_unlabled

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size))  # 32000 // 128 = 250

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size:(i + 1) * self.batch_size] # [0:128], [128:256]
                X, y = self.__data_generation(batch_ids)
                
                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)
        if self.shuffle:
            np.random.shuffle(indexes)
        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.num_unlabled // 2)
        X_l = l.reshape(self.num_unlabled // 2, 1, 1, 1)
        y_l = l.reshape(self.num_unlabled // 2, 1)

        X_labeled = self.X_train[batch_ids[:(self.batch_size - self.num_unlabled)]]
        X1 = self.X_train[batch_ids[(self.batch_size - self.num_unlabled):(self.batch_size - self.num_unlabled) + self.num_unlabled // 2]]
        X2 = self.X_train[batch_ids[((self.batch_size - self.num_unlabled) + self.num_unlabled // 2):]]
        X_1 = X1 * X_l + X2 * (1 - X_l)
        X_2 = X2 * X_l + X1 * (1 - X_l)
        X_unlabeled = np.concatenate((X_1, X_2), axis=0)
        X = np.concatenate((X_labeled, X_unlabeled), axis=0)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])
          

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y_labeled = y_train_[batch_ids[:(self.batch_size - self.num_unlabled)]]
                y1 = y_train_[batch_ids[(self.batch_size - self.num_unlabled):(self.batch_size - self.num_unlabled) + self.num_unlabled // 2]]
                y2 = y_train_[batch_ids[((self.batch_size - self.num_unlabled) + self.num_unlabled // 2):]]
                y_1 = y1 * y_l + y2 * (1 - y_l)
                y_2 = y2 * y_l + y1 * (1 - y_l)
                y_unlabeled = np.concatenate((y_1, y_2), axis=0)
                y.append(np.concatenate((y_labeled, y_unlabeled), axis=0))
        else:
            y_labeled = self.y_train[batch_ids[:(self.batch_size - self.num_unlabled)]]
            y1 = self.y_train[batch_ids[(self.batch_size - self.num_unlabled):(self.batch_size - self.num_unlabled) + self.num_unlabled // 2]]
            y2 = self.y_train[batch_ids[((self.batch_size - self.num_unlabled) + self.num_unlabled // 2):]]
            y_1 = y1 * y_l + y2 * (1 - y_l)
            y_2 = y2 * y_l + y1 * (1 - y_l)
            y_unlabeled = np.concatenate((y_1, y_2), axis=0)
            y = np.concatenate((y_labeled, y_unlabeled), axis=0)

        return X, y

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

In [0]:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

In [0]:
training_generator = MixupGenerator(X_train, y_train, batch_size=128, alpha=0.2, datagen=datagen)()
model.fit_generator(generator=training_generator,
                    
                    steps_per_epoch=250,  # 32000 // 128
                    epochs=100,
                    validation_data=(X_test, y_test),
                    verbose=1)

Epoch 1/100
250/250 [==============================] - 14s 57ms/step - loss: 1.8222 - acc: 0.3376 - val_loss: 1.4347 - val_acc: 0.4909
Epoch 2/100
250/250 [==============================] - 13s 53ms/step - loss: 1.5045 - acc: 0.4686 - val_loss: 1.2789 - val_acc: 0.5338
Epoch 3/100
250/250 [==============================] - 13s 53ms/step - loss: 1.3556 - acc: 0.5283 - val_loss: 1.1200 - val_acc: 0.5972
Epoch 4/100
250/250 [==============================] - 13s 53ms/step - loss: 1.2435 - acc: 0.5701 - val_loss: 1.1359 - val_acc: 0.6024
Epoch 5/100
250/250 [==============================] - 13s 53ms/step - loss: 1.1877 - acc: 0.5952 - val_loss: 0.9694 - val_acc: 0.6620
Epoch 6/100
250/250 [==============================] - 13s 53ms/step - loss: 1.1309 - acc: 0.6178 - val_loss: 0.9822 - val_acc: 0.6589
Epoch 7/100
250/250 [==============================] - 13s 54ms/step - loss: 1.0885 - acc: 0.6344 - val_loss: 0.9020 - val_acc: 0.6853
Epoch 8/100
250/250 [==============================] - 